#### Importing Required Libraries

In [3]:
from numpy import array
import pandas as pd
import numpy as np
from datetime import datetime
from numpy import asarray
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, KFold
import random
from sklearn.utils import shuffle
import tensorflow as tf
import pickle
import csv
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

#### Reading historical data from multiple files and cleaning and formatting the data

In [4]:
# Reading CPI CPIH data from CSV file
cpi_monthly_1950_1988 = pd.read_csv('../input_data/cpi_data_Jan1950_Dec1988.csv')
cpi_monthly_1989_2023 = pd.read_csv('../input_data/cpi_data_Jan1989_Oct2023.csv')

#Concatinaing  both the dataframes
cpi_data = pd.concat([cpi_monthly_1950_1988, cpi_monthly_1989_2023], axis=0)

# Calculating the Average CPIH from 1950 to 2023
avglist = cpi_data[['CPI', 'CPIH']].mean(axis=0).tolist()
average = round(sum(avglist)/len(avglist),1)
cpi_data['Average CPIH'] = average

# Converting the 'Dates' column in the cpi_data DataFrame to datetime format
cpi_data['Dates'] = pd.to_datetime(cpi_data['Dates'])
print(cpi_data.head())

       Dates  CPIH  CPI  Average CPIH
0 1950-01-01   4.3  4.1           4.6
1 1950-02-01   4.3  4.1           4.6
2 1950-03-01   4.8  4.7           4.6
3 1950-04-01   5.4  5.5           4.6
4 1950-05-01   3.7  3.7           4.6
        Year  Change in %
0 1949-01-01          3.3
1 1950-01-01          3.3
2 1951-01-01          3.7
3 1952-01-01          1.5
4 1953-01-01          5.6


In [23]:
# Reading GDP data from CSV file
gdp_yearly_1949_2022 =  pd.read_csv('../input_data/gdp_yearly_1949_2022.csv')
gdp_quarterly_1955_2023 =  pd.read_csv('../input_data/gdp_quarterly_1955_2023.csv')

# Transforming 'Year' column in gdp_yearly_1949_2022 to datetime format
gdp_yearly_1949_2022['Month'] = '-01-01'
gdp_yearly_1949_2022['Year'] = gdp_yearly_1949_2022['Year'].astype('str')+gdp_yearly_1949_2022['Month'].astype('str')
gdp_yearly_1949_2022 = gdp_yearly_1949_2022.drop(columns=['Month'])
gdp_yearly_1949_2022['Year'] = pd.to_datetime(gdp_yearly_1949_2022['Year'])
print(gdp_yearly_1949_2022.head())

print(gdp_quarterly_1955_2023.head())
# Convert 'Dates' column to "YYYY-MM-DD" format
def convert_to_date(quarter_year):
    year, quarter = quarter_year.split()
    month = (int(quarter[1:]) - 1) * 3 + 1  # Convert quarter to month
    return f'{year}-{month:02d}-01'

gdp_quarterly_1955_2023['Dates'] = gdp_quarterly_1955_2023['Dates'].apply(convert_to_date)
print(gdp_quarterly_1955_2023.head())

        Year  Change in %
0 1949-01-01          3.3
1 1950-01-01          3.3
     Dates  GDP Change in %
0  1955 Q2              0.0
1  1955 Q3              2.0
        Dates  GDP Change in %
0  1955-04-01              0.0
1  1955-07-01              2.0


#### Function to transform a time series sequence into a supervised learning problem with input-output pairs (X, y).

In [5]:
def split_sequences(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence) - (n_steps_in + n_steps_out)):
        append_X = []
        append_y = []
        for j in range(n_steps_in):
            append_X.append(sequence[i + j])
        for k in range(n_steps_out):
            append_y.append(sequence[i + n_steps_in + k + 1])

        X.append(append_X)
        y.append(append_y)

    return np.array(X), np.array(y)

#### Function to prepare and scale time series data, create sequences, and split data for training and testing.

In [6]:
def predict_inflation(df, target_column, n_steps_in, n_steps_out):
    # Scale data between 0 and 1
    scaler = MinMaxScaler()
    scaled_data = asarray(df[target_column]).reshape(-1, 1)
    scaled_data = scaler.fit_transform(scaled_data)
    # Omit the last 50 observations for out-of-sample forecast
    out_of_sample_input = scaled_data[len(scaled_data) - 50 :, 0]
    scaled_data = scaled_data[: len(scaled_data) - 50, 0]


    # Create sequences for input (X) and output (y)
    X, y = split_sequences(scaled_data, n_steps_in, n_steps_out)


    # Split data into training and testing 80:20
    total_rows = X.shape[0]
    train_rows = int(total_rows * 0.8)
    test_rows = total_rows - train_rows


    # Obtain training and testing data
    X_train = X[:train_rows]
    X_test = X[train_rows:]
    y_train = y[:train_rows]
    y_test = y[train_rows:]


    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], 1)

    # Build and train the LSTM model
    model = Sequential()
    model.add(
        LSTM(75, activation="relu", return_sequences=True, input_shape=(n_steps_in, 1))
    )
    model.add(LSTM(75, activation="relu", return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(n_steps_out))
    model.compile(loss="mse", optimizer="adam")
    training_model = model.fit(X_train, y_train, epochs=50, verbose=1)


    # Perform out-of-sample forecast
    out_of_sample_input = asarray(out_of_sample_input).reshape(1, n_steps_in)
    out_of_sample_forecast = model.predict(out_of_sample_input, verbose=0)


    # Inversely scale the forecasted data
    list_forecast = scaler.inverse_transform(out_of_sample_forecast).tolist()
    print(
        f"Predicted {target_column} Data for next {n_steps_out} months: ",
        list_forecast[0],
    )
    return list_forecast

In [7]:
input_df = cpi_data.copy()
input_df.set_index("Dates", inplace=True)


# Predict CPI for the next 12 months
cpi_predicted_list = predict_inflation(input_df, "CPI", 50, 12)


# Predict CPIH for the next 12 months
cpih_predicted_list = predict_inflation(input_df, "CPIH", 50, 12)



Epoch 1/50

20/20 [==============================] - 5s 84ms/step - loss: 0.0589
Epoch 2/50
20/20 [==============================] - 2s 80ms/step - loss: 0.0214
Epoch 3/50
20/20 [==============================] - 2s 77ms/step - loss: 0.0171
Epoch 4/50
20/20 [==============================] - 2s 76ms/step - loss: 0.0164
Epoch 5/50
20/20 [==============================] - 2s 85ms/step - loss: 0.0158
Epoch 6/50
20/20 [==============================] - 2s 82ms/step - loss: 0.0138
Epoch 7/50
20/20 [==============================] - 2s 83ms/step - loss: 0.0141
Epoch 8/50
20/20 [==============================] - 2s 82ms/step - loss: 0.0131
Epoch 9/50
20/20 [==============================] - 2s 80ms/step - loss: 0.0121
Epoch 10/50
20/20 [==============================] - 2s 81ms/step - loss: 0.0123
Epoch 11/50
20/20 [==============================] - 2s 81ms/step - loss: 0.0115
Epoch 12/50
20/20 [==============================] - 2s 84ms/step - loss: 0.0120
Epoch 13/50
20/20 [===============

#### Calculate and print error metrics for CPI and CPIH predictions based on the last forecasted values

In [8]:
# Calculate errors for CPI
mse_cpi = mean_squared_error(cpi_data["CPI"].tolist()[-len(cpi_predicted_list[0]):], cpi_predicted_list[0])
rmse_cpi = math.sqrt(mse_cpi)
mae_cpi = mean_absolute_error(cpi_data["CPI"].tolist()[-len(cpi_predicted_list[0]):], cpi_predicted_list[0])


# Calculate errors for CPIH
mse_cpih = mean_squared_error(cpi_data["CPIH"].tolist()[-len(cpih_predicted_list[0]):], cpih_predicted_list[0])
rmse_cpih = math.sqrt(mse_cpih)
mae_cpih = mean_absolute_error(cpi_data["CPIH"].tolist()[-len(cpih_predicted_list[0]):], cpih_predicted_list[0])


# Print the calculated error metrics
print(f"CPI Errors: MSE = {mse_cpi}, RMSE = {rmse_cpi}, MAE = {mae_cpi}")
print(f"CPIH Errors: MSE = {mse_cpih}, RMSE = {rmse_cpih}, MAE = {mae_cpih}")

CPI Errors: MSE = 10.251372543195584, RMSE = 3.201776466775216, MAE = 2.6887910207112626
CPIH Errors: MSE = 8.199690470644937, RMSE = 2.863510165975483, MAE = 2.5444024562835694


#### Creating CSV file of Actual Historical CPI and CPIH Data

In [11]:
# Formating Dates column data.
cpi_data['Dates'] = pd.to_datetime(cpi_data['Dates']).dt.strftime('%Y-%m-%d')
# Saving historical data in output folder
cpi_data.to_excel('../output_data/CPI_CPIH_Inflation_Historical_data.xlsx', index=False)
cpi_data.head()
cpi_data.tail()

,Dates,CPIH,CPI,Average CPIH
413,2023-06-01,7.3,7.9,4.6
414,2023-07-01,6.4,6.8,4.6
415,2023-08-01,6.3,6.7,4.6
416,2023-09-01,6.3,6.7,4.6
417,2023-10-01,4.7,4.6,4.6


In [12]:
# Assuming 'prediction_start_date' is the base date, starting from November 2023
prediction_start_date = datetime.strptime('11/2023', '%m/%Y')
# Specified the 12 number becuase we want months future months list
month = 12
# Create the list of datetime objects
future_date_list = [prediction_start_date + relativedelta(months=i) for i in range(month)]
# Creating Final Dataframe of predicted CPI and CPIH data
prediction_df = pd.DataFrame({'Dates':future_date_list, 'CPI':cpi_predicted_list[0], 'CPIH':cpih_predicted_list[0]})
prediction_df['Average CPIH'] = average
prediction_df[['CPI', 'CPIH', 'Average CPIH']] = prediction_df[['CPI', 'CPIH', 'Average CPIH']].round(2)
prediction_df.to_excel('../output_data/CPI_CPIH_Inflation_Over_Next_12_Months.xlsx', index=False)
prediction_df.head()

,Dates,CPI,CPIH,Average CPIH
0,2023-11-01,5.23,5.29,4.6
1,2023-12-01,5.52,5.11,4.6
2,2024-01-01,5.73,5.07,4.6
3,2024-02-01,6.08,5.32,4.6
4,2024-03-01,6.54,5.24,4.6


#### Implementation for GDP

##### existing code 

In [39]:
#### Bulding a model for GDP Growth using LSTM time varient approach and training the model on historical data
def predict_gdp_growth(gdp_data, target_column, n_steps_in, n_steps_out):
    # Scale data between -1 and 1
    gdp_scaler = MinMaxScaler()
    scaled_gdp = asarray(gdp_data[target_column]).reshape(-1, 1)
    scaled_gdp = gdp_scaler.fit_transform(scaled_gdp)
    # Omit the last 50 observations for out-of-sample forecast
    gdp_sample_forecast_input = scaled_gdp[len(scaled_gdp) - n_steps_in:, 0]
    scaled_gdp = scaled_gdp[:, 0]

    # Set the number of lags and forecast periods
    # n_steps_in = 10
    # n_steps_out = 5
    
    # Create sequences for input (X) and output (y)
    X, y = split_sequences(scaled_gdp, n_steps_in, n_steps_out)
    
    # Split data into training and testing 80:20
    total_rows = X.shape[0]
    train_rows = int(total_rows * 0.8)
    test_rows = total_rows - train_rows
    
    # Obtain training and testing data
    X_train = X[:train_rows]
    X_test = X[train_rows:]
    y_train = y[:train_rows]
    y_test = y[train_rows:]
    
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], 1)
    
    # Build and train the LSTM model
    
    # Initialize a sequential model
    model = Sequential()
    
    # Add the first LSTM layer with 75 units, ReLU activation, and input shape (n_steps_in, 1)
    # Return sequences as there is another LSTM layer following
    model.add(LSTM(75, activation='relu', input_shape=(n_steps_in, 1)))
    
    # Add a dropout layer with a dropout rate of 0.2 to prevent overfitting
    model.add(Dropout(0.5))
    
    # Add a dense (fully connected) layer with n_steps_out units for prediction
    model.add(Dense(n_steps_out))
    
    # Compile the model with mean squared error loss and the Adam optimizer
    model.compile(loss='mse', optimizer='adam')
    
    training_model = model.fit(X_train, y_train, epochs=50, verbose=1)
    
    # Perform out-of-sample forecast
    gdp_sample_forecast_input = asarray(gdp_sample_forecast_input).reshape(1, n_steps_in)
    dpg_sample_forecast = model.predict(gdp_sample_forecast_input, verbose=0)
    
    # Inversely scale the forecasted data and save it to a CSV file
    gdp_list_forecast = gdp_scaler.inverse_transform(dpg_sample_forecast).tolist()
    # print('Predicted GDP Growth for next 10 Years: ', gdp_list_forecast[0])
    return gdp_list_forecast[0]

In [40]:
# Setting idex as Dates column
gdp_input_df = gdp_yearly_1949_2022.copy()
gdp_input_df.set_index('Year', inplace=True)
gdp_predicted_list = predict_gdp_growth(gdp_input_df, 'Change in %', 10, 5)

Epoch 1/50
2/2 [==============================] - 2s 38ms/step - loss: 0.4569
Epoch 2/50
2/2 [==============================] - 0s 16ms/step - loss: 0.4312
Epoch 3/50
2/2 [==============================] - 0s 15ms/step - loss: 0.4033
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 0.3741
Epoch 5/50
2/2 [==============================] - 0s 40ms/step - loss: 0.3432
Epoch 6/50
2/2 [==============================] - 0s 33ms/step - loss: 0.3151
Epoch 7/50
2/2 [==============================] - 0s 23ms/step - loss: 0.2844
Epoch 8/50
2/2 [==============================] - 0s 16ms/step - loss: 0.2661
Epoch 9/50
2/2 [==============================] - 0s 17ms/step - loss: 0.2259
Epoch 10/50
2/2 [==============================] - 0s 16ms/step - loss: 0.1989
Epoch 11/50
2/2 [==============================] - 0s 25ms/step - loss: 0.1795
Epoch 12/50
2/2 [==============================] - 0s 24ms/step - loss: 0.1308
Epoch 13/50
2/2 [==============================] - 0s 24ms/st

In [41]:
gdp_quarterly_input_df = gdp_quarterly_1955_2023.copy()
gdp_quarterly_input_df.set_index('Dates', inplace=True)
gdp_quarterly_predicted_list = predict_gdp_growth(gdp_quarterly_input_df, 'GDP Change in %', 50, 12)

Epoch 1/50
6/6 [==============================] - 2s 31ms/step - loss: 0.3058
Epoch 2/50
6/6 [==============================] - 0s 28ms/step - loss: 0.2630
Epoch 3/50
6/6 [==============================] - 0s 28ms/step - loss: 0.2158
Epoch 4/50
6/6 [==============================] - 0s 28ms/step - loss: 0.1386
Epoch 5/50
6/6 [==============================] - 0s 27ms/step - loss: 0.0915
Epoch 6/50
6/6 [==============================] - 0s 39ms/step - loss: 0.0762
Epoch 7/50
6/6 [==============================] - 0s 27ms/step - loss: 0.0622
Epoch 8/50
6/6 [==============================] - 0s 26ms/step - loss: 0.0518
Epoch 9/50
6/6 [==============================] - 0s 27ms/step - loss: 0.0476
Epoch 10/50
6/6 [==============================] - 0s 26ms/step - loss: 0.0423
Epoch 11/50
6/6 [==============================] - 0s 29ms/step - loss: 0.0366
Epoch 12/50
6/6 [==============================] - 0s 36ms/step - loss: 0.0345
Epoch 13/50
6/6 [==============================] - 0s 27ms/st

In [42]:
# Historic data
gdp_quarterly_1955_2023.to_excel('../output_data/GDP_Growth_Historical_Querterly_data.xlsx', index=False)
print(gdp_quarterly_1955_2023.head())

# Generate a list of the next 5 years
quarters = 12
# Querts should be 01/XXXX, 04/XXXX, 07/XXXX, 10/XXXX months
gdp_growth_start_quarter = datetime.strptime('10/2023', '%m/%Y')
next_12_quarters = [gdp_growth_start_quarter + relativedelta(months=3 * i) for i in range(quarters)]
predicted_querterly_gdp_growth = pd.DataFrame({'Year':next_12_quarters, 'GDP Change in %':gdp_quarterly_predicted_list})
predicted_querterly_gdp_growth['GDP Change in %'] = predicted_querterly_gdp_growth['GDP Change in %'].round(2)
predicted_querterly_gdp_growth.to_excel('../output_data/GDP_Growth_Over_next_12_Querters.xlsx', index=False)
print(predicted_querterly_gdp_growth)

,Year,GDP Change in %
0,2023-10-01,-0.12
1,2024-01-01,0.20
2,2024-04-01,-0.39
3,2024-07-01,0.35
4,2024-10-01,-0.30
5,2025-01-01,0.08
6,2025-04-01,0.18
7,2025-07-01,-0.34
8,2025-10-01,0.26
9,2026-01-01,0.16


In [43]:
# Historic data
gdp_yearly_1949_2022.to_excel('../output_data/GDP_Growth_Historical_data.xlsx', index=False)
print(gdp_yearly_1949_2022.head())

# Generate a list of the next 5 years
years = 5
gdp_growth_start_year = datetime.strptime('01/2023', '%m/%Y')
next_5_years = [gdp_growth_start_year + relativedelta(years=i) for i in range(years)]
predicted_gdp_growth = pd.DataFrame({'Year':next_5_years, 'Change in %':gdp_predicted_list})
predicted_gdp_growth['Change in %'] = predicted_gdp_growth['Change in %'].round(2)
predicted_gdp_growth.to_excel('../output_data/GDP_Growth_Over_next_5_Years.xlsx', index=False)
print(predicted_gdp_growth)

,Year,Change in %
0,2023-01-01,0.67
1,2024-01-01,0.97
2,2025-01-01,0.05
3,2026-01-01,0.76
4,2027-01-01,0.45


In [17]:
def predict_time_series(df, n_steps_in, n_steps_out, target_column=None):
    # Scale data between 0 and 1
    scaler = MinMaxScaler()
    
    # If target_column is specified, use it; otherwise, use the first column
    target_column = target_column or df.columns[0]
    
    scaled_data = asarray(df[target_column]).reshape(-1, 1)
    scaled_data = scaler.fit_transform(scaled_data)
    
    # Omit the last 50 observations for out-of-sample forecast
    out_of_sample_input = scaled_data[len(scaled_data) - 50 :, 0]
    scaled_data = scaled_data[: len(scaled_data) - 50, 0]

    # Create sequences for input (X) and output (y)
    X, y = split_sequences(scaled_data, n_steps_in, n_steps_out)

    # Split data into training and testing 80:20
    total_rows = X.shape[0]
    train_rows = int(total_rows * 0.8)
    test_rows = total_rows - train_rows

    # Obtain training and testing data
    X_train = X[:train_rows]
    X_test = X[train_rows:]
    y_train = y[:train_rows]
    y_test = y[train_rows:]

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], 1)

    # Build and train the LSTM model
    model = Sequential()
    model.add(
        LSTM(75, activation="relu", return_sequences=True, input_shape=(n_steps_in, 1))
    )
    model.add(LSTM(75, activation="relu", return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(n_steps_out))
    model.compile(loss="mse", optimizer="adam")
    training_model = model.fit(X_train, y_train, epochs=50, verbose=1)

    # Perform out-of-sample forecast
    out_of_sample_input = asarray(out_of_sample_input).reshape(1, n_steps_in)
    out_of_sample_forecast = model.predict(out_of_sample_input, verbose=0)

    # Inversely scale the forecasted data
    list_forecast = scaler.inverse_transform(out_of_sample_forecast).tolist()
    print(
        f"Predicted {target_column} Data for next {n_steps_out} months: ",
        list_forecast[0],
    )
    return list_forecast